In [1]:
# Load the Tensorboard notebook extension
%load_ext tensorboard

In [2]:
import tensorflow as tf
import datetime

In [3]:
# Clear any logs from previous run
!rm -rf ./logs/

In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11493376/11490434 [==============================] - 1s 0us/step


In [9]:
def create_model():
    return tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

In [11]:
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
log_dir = 'logs/fit/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
model.fit(x=x_train,
          y=y_train,
          epochs=5,
          validation_data=(x_test, y_test),
          callbacks=[tensorboard_callback])

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 5s 88us/sample - loss: 0.2188 - accuracy: 0.9352 - val_loss: 0.1155 - val_accuracy: 0.9650
Epoch 2/5
60000/60000 [==============================] - 5s 77us/sample - loss: 0.0952 - accuracy: 0.9712 - val_loss: 0.0812 - val_accuracy: 0.9738
Epoch 3/5
60000/60000 [==============================] - 5s 80us/sample - loss: 0.0684 - accuracy: 0.9783 - val_loss: 0.0693 - val_accuracy: 0.9787
Epoch 4/5
60000/60000 [==============================] - 5s 83us/sample - loss: 0.0540 - accuracy: 0.9829 - val_loss: 0.0660 - val_accuracy: 0.9795
Epoch 5/5
60000/60000 [==============================] - 5s 78us/sample - loss: 0.0428 - accuracy: 0.9868 - val_loss: 0.0648 - val_accuracy: 0.9820


In [12]:
%tensorboard --logdir logs/fit

When training with methods such as `tf.GradientTape()`, use `tf.summary` to log the required information.

Use the same dataset as above, but convert it to `tf.data.Dataset` to take advantage of baching capabilities:

In [13]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_dataset = train_dataset.shuffle(60000).batch(64)
test_dataset = test_dataset.batch(64)

In [14]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

Create stateful metrics that can be used to accumulate values during training and logged at any point:

In [15]:
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [16]:
def train_step(model, optimizer, x_train, y_train):
    with tf.GradientTape() as tape:
        predictions = model(x_train, training=True)
        loss = loss_object(y_train, predictions)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(y_train, predictions)

In [17]:
def test_step(model, x_test, y_test):
    predictions = model(x_test)
    loss = loss_object(y_test, predictions)
    
    test_loss(loss)
    test_accuracy(y_test, predictions)

Set up summary writers to write the summaries to disk in a different logs directory:

In [18]:
current_time = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

Start training. Use `tf.summary.scalar()` to log metrics (loss and accuracy) during training/testing within the scope of the summary writers to write the summaries to disk. You have control over which metrics to log and how often to do it. Other `tf.summary` fuctnions enable logging other types of data.

In [19]:
model = create_model()

EPOCHS = 5

for epoch in range(EPOCHS):
    for (x_train, y_train) in train_dataset:
        train_step(model, optimizer, x_train, y_train)
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)
    
    for (x_test, y_test) in test_dataset:
        test_step(model, x_test, y_test)
    with test_summary_writer.as_default():
        tf.summary.scalar('loss', test_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)
    
    template = 'Epoch: {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))
    # Reset metrics every epoch
    train_loss.reset_states()
    test_loss.reset_states()
    train_accuracy.reset_states()
    test_accuracy.reset_states()



If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch: 1, Loss: 0.240592360496521, Accuracy: 93.07167053222656, Test Loss: 0.11689313501119614, Test Accuracy: 96.41999816894531
Epoch: 2, Loss: 0.17185638844966888, Accuracy: 95.00333404541016, Test Loss: 0.0985279455780983, Test Accuracy: 96.94499969482422
Epoch: 3, Loss: 0.1388174444437027, Accuracy: 95.9316635131836, Test Loss: 0.08924048393964767, Test Accuracy: 97.23332977294922
Epoch: 4, Loss: 0.11813043802976608, Accuracy: 96.5183334350586, Test Loss: 0.08318296074867249, Test Accuracy: 97.41999

In [20]:
%tensorboard --logdir logs/gradient_tape